In [1]:
# generalized model loading

import torch
from GenericMlUtils import *

print('load model...')

if False:
    pass
elif False:
    modelName = "bigscience/bloomz-7b1" # recommended for instruction prompting in non-english
    
    # code from https://huggingface.co/bigscience/bloomz-7b1-p3
    from transformers import AutoModelForCausalLM, AutoTokenizer
    
    m = ModelBundle()
    m.tokenizer = AutoTokenizer.from_pretrained(modelName)
    m.model = AutoModelForCausalLM.from_pretrained(modelName, torch_dtype="auto", device_map="auto")

    #inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt").to("cuda")
    #outputs = model.generate(inputs)
    #print(tokenizer.decode(outputs[0]))
    
    pass
elif True: # load galactica
    from transformers import AutoTokenizer, OPTForCausalLM

    modelName = "facebook/galactica-30b" # to big, explodes GPU
    modelName = "facebook/galactica-6.7b"

    # code is from https://github.com/paperswithcode/galai
    m = ModelBundle()
    m.model = OPTForCausalLM.from_pretrained(modelName, device_map="auto")
    #m.model = OPTForCausalLM.from_pretrained(modelName, device_map="auto",   load_in_8bit=True)
    
    #model = OPTForCausalLM.from_pretrained(modelName, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.bfloat16)
    m.tokenizer = AutoTokenizer.from_pretrained(modelName)


elif True: # load opt
    from transformers import AutoModelForCausalLM, AutoTokenizer

    m = ModelBundle()
    
    #modelName = "facebook/opt-30b"
    modelName = "facebook/opt-13b"
    

    # from https://huggingface.co/facebook/opt-30b

    m.model = AutoModelForCausalLM.from_pretrained(modelName, torch_dtype=torch.float16)
    m.model = m.model.cuda()

    # the fast tokenizer currently does not work correctly

    m.tokenizer = AutoTokenizer.from_pretrained(modelName, use_fast=False)


load model...


Downloading:   0%|          | 0.00/166 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [4]:
# helper to open a file and search for a keyword in it, return all lines which match
def z88844(path, searchStrings):
    f = open(path, 'r')
    lines = f.readlines()
    f.close()
    
    hitLines = []
    
    for iLine in lines:
        if iLine[-1] == '\n':
            iLine = iLine[:-1]
        
        
        # remove new style seperator between raw text and reformulated text
        idx1 = iLine.find(' #%%^ ')
        if idx1 != -1:
            iLine = iLine[idx1+len(' #%%^ '):]
        
        iLine0 = iLine.lower() # convert to lower case for better search by keywords
        
        
        hitKeywordCnt = 0
        for iKeyword in searchStrings:
            if iLine0.find(iKeyword+' ') != -1:
                hitKeywordCnt = hitKeywordCnt+1
        
        if hitKeywordCnt == len(searchStrings):
            print(iLine)
            hitLines.append(iLine)
    
    return hitLines


# searches and returns all lines which contain the mentioned keywords
def z673783(keywords):
    path = '.'
    import os
    filenames = os.listdir(path)

    # we only care about filenames which are entries of our database
    filenames = list(filter(lambda iv: iv.find('out_')!=-1, filenames))

    #print(filenames) # DBG

    allHitlines = [] # strings of the lines which match up

    for iFilename in filenames:
        allHitlines = allHitlines+z88844(iFilename, keywords)
    
    return allHitlines



In [1]:
questionTxt0 = 'What is AGI?' # 15.12.2022    finds correct answer

# extracts keywords of a question
def extractKeywords(questionTxt):
    questionTxt0 = questionTxt.lower()
    questionTxt0 = questionTxt0.replace('?', ' ? ')
    
    # stemming
    questionTxt0 = questionTxt0.replace('dealing', 'deals')
    
    tokens = questionTxt0.split(' ')
    outKeywords = []
    for iToken in tokens:
        if iToken == '':
            continue
        
        blacklist = ['is', 'what', 'with', '?', 'who', 'when', 'a']
        if iToken in blacklist:
            continue # ignore token as keyword
        
        outKeywords.append(iToken)
    
    return outKeywords

keywords = extractKeywords(questionTxt0)


allHitlines = []

# HACK< we try definitions with 'is a' first, take the results if it returns any > 
keywords1 = keywords + ['is a']
if len(allHitlines) == 0: # didnt find any answers yet?
    # search in database of facts for all lines which match the keywords as best as possible
    print(str(keywords1))
    allHitlines = z673783(keywords1)

keywords1 = keywords + ['is an']
if len(allHitlines) == 0: # didnt find any answers yet?
    # search in database of facts for all lines which match the keywords as best as possible
    print(str(keywords1))
    allHitlines = z673783(keywords1)


# else we backup to search by taking keywords into account
if len(allHitlines) == 0:
    # search in database of facts for all lines which match the keywords as best as possible
    allHitlines = z673783(keywords)





# process lines and get rid of knowledge system idocracies
allHitlines2 = []
for iLine in allHitlines:
    if iLine.endswith('? YES'): # is of a statement form
        allHitlines2.append(iLine[:-5]+'.') # add line without YES as statement



# compose eligable lines 
eligableKnowledgeLines = '\n'.join(allHitlines2)


# purpose: answer side for Q&A system
# model bigscience/bloom-7b1     result: gives the CORRECT answer!
# model galactica-6.7b     result: gives the CORRECT answer!
prompt = """"Statistics deals with the planning of data collection in terms of the design of surveys and experiments.

Q: What is statistics dealing with?
A:"""


prompt = f"""{eligableKnowledgeLines}

Q: {questionTxt0}
A:"""



res0 = None

config = InferenceConfig()
config.doSample = False
config.topP = 0.31
print('\n\n\n\n')
print(prompt)
if True: # run prompt?
    print('~run')
    res0 = inf(m, prompt, 300, config)[1:]
    print('~done')
    print('===')
    print(res0)

# process answer to extract relevant parts
res1 = res0[1:]


res2 = None 
idx0 = res1.find('\nQ:')
if idx0 != -1:
    res2 = res1[:idx0]

# 'res2' is the answer to the question
print('ANSWER:')
print(res2)

['agi', 'is a']


NameError: name 'z673783' is not defined